In [1]:
import json
import random
import string
import datetime
import re
import pandas as pd
from websocket import create_connection

In [2]:
def gen_session(prefix="cs"):
    return prefix + "_" + "".join(random.choices(string.ascii_lowercase, k=12))


def prepend_header(msg):
    return f"~m~{len(msg)}~m~{msg}"


def create_message(method, params):
    return json.dumps({"m": method, "p": params}, separators=(",", ":"))


def parse_series(raw_data):
    try:
        out = re.search('"s":\[(.+?)\}\]', raw_data).group(1)
        x = out.split(',{"')
        data = []
        for xi in x:
            xi = re.split("\[|:|,|\]", xi)
            ts = datetime.datetime.fromtimestamp(float(xi[4]))
            row = [ts] + [float(xi[i]) for i in range(5, 10)]
            data.append(row)
        df = pd.DataFrame(data, columns=["datetime", "open", "high", "low", "close", "volume"])
        df.set_index("datetime", inplace=True)
        return df
    except Exception as e:
        print("Parse error:", e)
        return pd.DataFrame()  # return empty DF on error

In [27]:

filePath = r"./files/forex_key.json"
with open(filePath, "r") as file:
        infor = json.load(file)
        token = infor["tradingview"]["sen07"]["token"]
symbol = "EURUSD"
source = "OANDA"
timeframe = "1"  # 1m
n_bars = 20000

# Replace this with your sessionid cookie from TradingView
chart_session = gen_session("cs")
quote_session = gen_session("qs")

ws = create_connection("wss://data.tradingview.com/socket.io/websocket", timeout=5)

def send(method, params):
    msg = prepend_header(create_message(method, params))
    ws.send(msg)

# 1. Auth
send("set_auth_token", [token])

# 2. Create sessions
send("chart_create_session", [chart_session, ""])
send("quote_create_session", [quote_session])
send("quote_add_symbols", [quote_session, f"{source}:{symbol}", {"flags": ["force_permission"]}])

# 3. Resolve symbol & request data
send("resolve_symbol", [
    chart_session, "symbol_1",
    f'={{"symbol":"{source}:{symbol}","adjustment":"splits","session":"regular"}}'
])
send("create_series", [chart_session, "s1", "s1", "symbol_1", timeframe, n_bars])

print(f"Lấy dữ liệu nến {symbol} khung {timeframe}...")

raw = ""
while True:
    try:
        res = ws.recv()
        raw += res + "\n"
        if "series_completed" in res:
            break
    except Exception as e:
        print("Lỗi:", e)
        break

candles = parse_series(raw)

Lấy dữ liệu nến EURUSD khung 1...


In [28]:
chart_session

'cs_vmwycfcvleyw'

In [29]:
quote_session

'qs_vlglnobxhcdc'

In [30]:
candles

,open,high,low,close,volume
datetime,,,,,
2025-05-05 04:04:00,1.13062,1.13062,1.13062,1.13062,1.0
2025-05-05 04:05:00,1.13058,1.13058,1.13051,1.13051,2.0
2025-05-05 04:06:00,1.13059,1.13059,1.13038,1.13056,4.0
2025-05-05 04:07:00,1.13044,1.13057,1.13044,1.13057,2.0
2025-05-05 04:08:00,1.13044,1.13058,1.13044,1.13058,2.0
...,...,...,...,...,...
2025-05-10 03:55:00,1.12480,1.12490,1.12475,1.12489,57.0
2025-05-10 03:56:00,1.12488,1.12500,1.12481,1.12498,128.0
2025-05-10 03:57:00,1.12495,1.12506,1.12472,1.12483,439.0
